## Importing Required Libraries and dependencies.




In [4]:
#Import nessesary modules
import yfinance as yf
import pandas as pd
import json 
import numpy as np
import sqlite3
import streamlit as st

ImportError: dlopen(/Users/lexx/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pyarrow/lib.cpython-37m-darwin.so, 2): Symbol not found: _SSLCopyALPNProtocols
  Referenced from: /Users/lexx/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pyarrow/libarrow.1000.dylib
  Expected in: /System/Library/Frameworks/Security.framework/Versions/A/Security
 in /Users/lexx/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pyarrow/libarrow.1000.dylib

In [3]:
!pip install streamlit



## Step 1: Propt the user to customize choice for analysis

### 1.1 Prompt the user about how much capital they wish to invest??? Is it needed or we can calculate mean() for each dollar invested ?
### !!! Create widget on Streamlit for the prompt!!!

In [ ]:
#Ask user how much is the allocation for Equity Allocation
# Check the input 
while True:
    try:
        equity_allocation = float(input('Please enter the amount of capital you wish to invest :'))
        break
    except:
        print('Please enter a numerical value.')


## 1.2 Prompt the user how many years of historical data they wish to pull for the analysis.
### !!! Create widget on Streamlit for the prompt!!!

In [ ]:
# Prompt the user how many years of historical data they wish to pull for the analysis
# Check the input 
while True:
    try:
        num_years = int(input('Please enter the number of years (1-10) of historical data that you wish to retrieve for the analysis :'))
        break
    except:
        print('Please enter a numerical value.')


## 1.3 Prompt the user to select assets from each asset class.
### ??? Does the number of assets in each asset class should be the same for correct analysis???

### !!! Create widget on Streamlit for the prompt!!!

## Step 2 : 
### Data Collection and Cleaning
The first step is to collect data for the 3 different assets - crypto, commodities, and stocks. Using **yfinance** library for Pandas pull data from the Yahoo finance API. The data pulled for each asset class would have 10 tickers. Store the data for each asset class in separate Pandas DataFrames.

In [ ]:
# Define the lists of asset classes and their respective tickers. Set it as variables, so the selection can be changed easier if neded. 
### VARIATION 1
# crypto = ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD']
# stocks = ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC']
# commodities = ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']

# # Collect data for crypto asset class
# crypto_df = pd.DataFrame()
# for ticker in crypto:
#     crypto_df[ticker] = yf.download(ticker)['Close']

# # Collect data for stocks asset class
# stocks_df = pd.DataFrame()
# for ticker in stocks:
#     stocks_df[ticker] = yf.download(ticker)['Close']

# # Collect data for commodities asset class
# commodities_df = pd.DataFrame()
# for ticker in commodities:
#     commodities_df[ticker] = yf.download(ticker)['Close']


In [ ]:
# crypto_df.head(200)
# The problem with crypto historic data is taht diferent start times of the data(begining of the trading period is diffferent for each crypto)  

In [ ]:
### VARIATION 2
# Define the a dictionary of lists for each asset class
# api_pull = {'crypto' : ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD']
#             ,'stocks' : ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC']
#             ,'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']
           
#            }

# # Store data for each asset class in a dictionary
# data = {}
# for asset_class in api_pull:
#     for ticker in api_pull[asset_class]:
#         data[asset] = pd.DataFrame(yf.download(asset, start='2010-01-10', end ='2023-01-01'))

            

In [ ]:
# VARIATION 3
# Define the lists of asset classes and their respective tickers. Set it as variables, so the selection can be changed easier if neded.
api_pull = {'crypto' : ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD'],
            'stocks' : ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC'],
            'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']}

# Store data for each asset class in a dictionary
# Use .items()to iterate over the key-value pairs of the api_pull dictionary
# pull data for the number of years requested by the user
data = {}
for asset_class, tickers in api_pull.items():
    data[asset_class] = {}
    for ticker in tickers:
        data[asset_class][ticker] = yf.Ticker(ticker).history(period=f"{num_years}y")


In [ ]:
# Review pulled data:
data 

In [ ]:
# Create a data frame for each asset class with the close column
# Access the 'Close' column of the data for each asset class
# Extract the values of the data dictionary and then access the 'Close' column of each dataframe.
df_crypto = pd.DataFrame({ticker: data['crypto'][ticker]['Close'] for ticker in api_pull['crypto']})
df_stocks = pd.DataFrame({ticker: data['stocks'][ticker]['Close'] for ticker in api_pull['stocks']})
df_commodities = pd.DataFrame({ticker: data['commodities'][ticker]['Close'] for ticker in api_pull['commodities']})


In [ ]:
# Replace all the NaN values in a data frame using the fillna() method. Create a copy of the data frames.
df_crypto = df_crypto.fillna(0).copy()
df_stocks = df_stocks.fillna(0).copy()
df_commodities = df_commodities.fillna(0).copy()

In [ ]:
# Review first and last 10 rows for crypto assets class
display(df_crypto.head(10))
df_crypto.tail(10)



In [ ]:
# Review first and last 10 rows for stocks assets class
display(df_stocks.head(10))
df_stocks.tail(10)

In [ ]:
# Review first and last 10 rows for commodities assets class
display(df_commodities.head(10))
df_commodities.tail(10)

## Step 3 : 
### Prepare data ready for the Monte Carlo simulation
### ??? Should we prompt user to enter the number of simulations for the model ???
### !!! Keep in mind that for 'stocks' and 'comodities' trading_days = 252, but for 'crypto' trading_days = 365.!!!

In [ ]:
## Prepare the data for Monte Carlo simulation
returns = portfolio_data.pct_change() #Calculate returns for each asset
mean_return = returns.mean()
cov_matrix = returns.cov()

In [ ]:
# Set the number of simulations and trading days
num_simulations = 1000
stocks_n_comodities_num_trading_days = 252
crypto_num_trading_days = 365

## Step 4 : 
### Plot an overlay of the closing prices for each asset class (10 assetset each)
### ??? Plot the returns ???


In [ ]:

# Limit the number of securities for each alloocation( depends on the data fed)

# Get combination of alocated assets and weighting 
# Use KMeans algorithm for clustering 
#from sklearn.cluster import KMeans
#import numpy as np






In [ ]:
#Data from 3 assets with their weight amounts
#Limit each equity t0 allocate 10 items 
assets = {'equity':range(0,equity_alloc,10),'commodity':range(0,commodity_alloc,10),'crypto':range(0,crypto_alloc,10)}




In [ ]:
#Allowing only 3 cluster of assets 
kmeans = KMeans(n_clusters=3)



In [ ]:
#Fit the model to the asset data 
kmeans.fit(assets)



In [ ]:
# Plot vizualization fort the clusters 

#### For streamlit install pip install streamlit==0.62.0
### there is a problem problem was related to Python version.that pyarrow is a requirement in streamlit version 0.63.0. So you can use pip install streamlit==0.62.0 and it will work just fine